In [9]:
import pandas as pd
#Import full data set using Pandas:
data=pd.read_csv('/Users/yangxiang/A List/Uchicago/2020_Spring/NLP/Week_1/Food_Inspections.csv')
data.shape

(204389, 17)

In [10]:
#choose the roles only Results shows "fail"
data=data[data['Results']=='Fail']

##### 1.As in assignment 1, extract regulation descriptions from each record corresponding to a failed inspection

In [11]:
#Making sure that there are no NaNs in "Violations" column
data=data.dropna(subset=['Violations'])

In [4]:
#Using regular expression, parse "Violations" column to select only regulation descriptions
data=data.reset_index(drop=True)
import re
regulation_descriptions =data['Violations'].apply(lambda x: re.findall('[\d\s]+. [[A-Z\s\d\:\,\&\;\-\/]+Comments', x))

/Users/yangxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Possible nested set at position 10
  after removing the cwd from sys.path.


In [6]:
regulation_descriptions

0        [2. CITY OF CHICAGO FOOD SERVICE SANITATION CE...
1        [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
2        [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
3        [3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATU...
4        [22. PROPER COLD HOLDING TEMPERATURES - Commen...
5        [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
6        [5. PROCEDURES FOR RESPONDING TO VOMITING AND ...
7        [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
8        [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
9        [10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPL...
10       [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
11       [18. NO EVIDENCE OF RODENT OR INSECT OUTER OPE...
12       [3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATU...
13       [3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATU...
14       [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
15       [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...
16       [3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL .

In [5]:
import numpy as np
regulation_descriptions_total=np.concatenate(regulation_descriptions)

In [6]:
#Convert all regulation descriptions into one string 

# Function to convert   
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

In [7]:
regulation_descriptions_total=listToString(regulation_descriptions_total)

In [8]:
#Remove Numbers and "-Comments"
result = re.sub('\d+.\s|- Comments\s',  '',regulation_descriptions_total)

In [312]:
# regulation_descriptions_total1 = regulation_descriptions_total.replace("- Comments ", "")
# regulation_descriptions_total2 = regulation_descriptions_total1.replace("- Comments", "")

##### 2.Tokenize each regulation description

In [314]:
from nltk.tokenize import word_tokenize
regulation_descriptions_total_token=word_tokenize(result.lower())

##### 3.Find top-10 tokens (for the whole table)

In [315]:
from collections import Counter

In [316]:
Counter(regulation_descriptions_total_token).most_common(10)

[(',', 309444),
 ('and', 153646),
 (':', 117984),
 ('maintained', 72488),
 ('equipment', 63062),
 ('constructed', 59546),
 ('food', 57231),
 ('of', 55416),
 ('clean', 51859),
 ('installed', 49911)]

##### 4.Clean data: convert to lower case, remove stopwords, punctuation, numbers, etc

In [318]:
from nltk.corpus import stopwords
# Only return alphabetic string(strip tokens with numbers or punctuation)
tokens = [w for w in regulation_descriptions_total_token
         if w.isalpha()]
#Remove stopwords(useless words like and or ) in english 
no_stops = [t for t in tokens
           if t not in stopwords.words('english')]

In [ ]:
no_stops

##### 5.Find top-10 tokens again

In [319]:
Counter(no_stops).most_common(10)

[('maintained', 72488),
 ('equipment', 63062),
 ('constructed', 59546),
 ('food', 57231),
 ('clean', 51859),
 ('installed', 49911),
 ('properly', 49048),
 ('cleaning', 48220),
 ('surfaces', 39146),
 ('methods', 38228)]

#### 6.Find top-10 tokens after applying Porter stemming to the tokens obtained in step 4
###### PorterStemmer is known for its simplicity and speed. It is commonly useful in Information Retrieval Environments known as IR Environments for fast recall and fetching of search queries.

In [331]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
Porter_stemming=[ps.stem(w) for w in no_stops]

In [332]:
Counter(Porter_stemming).most_common(10)

[('clean', 121520),
 ('maintain', 76034),
 ('equip', 63062),
 ('food', 61652),
 ('construct', 59546),
 ('instal', 49911),
 ('properli', 49048),
 ('surfac', 39146),
 ('method', 38228),
 ('good', 37601)]

##### 7.Find top-10 tokens after applying Lancaster stemming to the tokens obtained in step 4.
LancasterStemmer is simple, but heavy stemming due to iterations and over-stemming may occur. Over-stemming causes the stems to be not linguistic, or they may have no meaning.

In [334]:
from nltk.stem import LancasterStemmer
lancaster=LancasterStemmer()
Lancaster_stemming=[lancaster.stem(w) for w in no_stops]

In [335]:
Counter(Lancaster_stemming).most_common(10)

[('cle', 124684),
 ('maintain', 76034),
 ('equip', 63062),
 ('food', 61652),
 ('construct', 59546),
 ('prop', 59390),
 ('instal', 49911),
 ('surfac', 39146),
 ('method', 38228),
 ('good', 37601)]

##### 8.Find top-10 tokens after applying lemmatization to the tokens obtained in step 4.

In [337]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lemmatized=[wordnet_lemmatizer.lemmatize(w) for w in no_stops]

In [338]:
Counter(lemmatized).most_common(10)

[('maintained', 72488),
 ('equipment', 63062),
 ('food', 61652),
 ('constructed', 59546),
 ('clean', 51859),
 ('installed', 49911),
 ('properly', 49048),
 ('cleaning', 48220),
 ('surface', 39146),
 ('method', 38228)]

##### 9.Compare top-10 tokens obtained in 3, 5, 6, 7, 8.

For top 10 that we obtained in 3, the data is extremely uncleaned, it have a lot of lower case, upper case problems, stopwords like"and","of" that does not really mean anything, useless punctuations etc, it is hard to generate useful information from it.

For top 10 that we obtained in 5, things are getting much better since i convert data to lower case, remove stopwords, punctuation, numbers, but same meaning words keep pop up on the list such as "clean" and "cleaning", the  Redundancy need to be reduced.

For top 10 that we obtained in 6, I applied Porter stemming Algorithm to stemmized some words, it was doen in quite short amount of time and significantly reduce the redundancy, but there are still some meaningless words such as 'instal','properli','equip' need to be edited so hunman being are able to read.

For top 10 that we obtained in 7, I applied Lancaster stemming Algorithm to stemmized some words, I found out that LancasterStemmer is simple, but heavy stemming due to iterations and over-stemming may occur. Over-stemming causes the stems to be not linguistic, or they may have no meaning. It strip the word even more and make it hard for human to read and understand

For top 10 that we obtained in 8, I applied lemmatization Algorithm to stemmized some words, this is the best result I have, the words makeing sense to me and every single results are the proper words, not some stem that does not have any meaning.

###### Here is the difference that I found between Stemming and Lemmatization from Datacamp.com

Stemming and Lemmatization both generate the root form of the inflected words. The difference is that stem might not be an actual word whereas, lemma is an actual language word.

Stemming follows an algorithm with steps to perform on the words which makes it faster. Whereas, in lemmatization, you used WordNet corpus and a corpus for stop words as well to produce lemma which makes it slower than stemming. You also had to define a parts-of-speech to obtain the correct lemma.

So when to use what! The above points show that if speed is focused then stemming should be used since lemmatizers scan a corpus which consumed time and processing. It depends on the application you are working on that decides if stemmers should be used or lemmatizers. If you are building a language application in which language is important you should use lemmatization as it uses a corpus to match root forms.